In [45]:
import dask.dataframe as dd
from typing import Tuple
df = dd.read_csv('Youtube0*.csv')

def get_lazy_shape(datf: dd) -> Tuple[int, int]: 
    return (df.shape[0].compute(), df.shape[1])

print(get_lazy_shape(df))
df.head()

(1956, 5)


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [46]:
#df["check" in df.CONTENT.compute().str.lower()].shape

def get_check(datf: dd, membership: int) -> int: 
    assert membership in [0,1]
    xs = ["check" in x for x in df.CONTENT.compute().str.lower()]
    ys =  [x==membership for x in df.CLASS.compute()]
    return sum([x and y for x,y in zip(xs,ys)])

def reporter(datf: dd) -> str: 
    c = df.CLASS.value_counts().compute()
    checkyes = get_check(datf, 1)
    checkno = get_check(datf, 0)
    s0 = f"The number of spam comments is {c.loc[1]}. \n"
    s1 = f"The number of non-spam comments is {c.loc[0]}. \n"
    s2 = "If the heuristic on the word check was the ONLY aspect of the model; \n"
    s3 = f"\twe'd have {c.loc[1] - checkyes} false negatives, \n"
    s4 = f"\tand {c.loc[0] - checkno} false positives. "
    
    return ''.join([s0, s1, s2, s3, s4])

print(reporter(df))

The number of spam comments is 1005. 
The number of non-spam comments is 951. 
If the heuristic on the word check was the ONLY aspect of the model; 
	we'd have 544 false negatives, 
	and 932 false positives. 



stopping Server...
